In [ ]:
import sys, os, re, math, sklearn, random
import numpy as np
import h5py as h5
import pandas as pd
from itertools import cycle
import matplotlib.pyplot as plt
import subprocess
import itertools

from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve, roc_curve, auc, accuracy_score
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import linear_model, svm, datasets
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from scipy.stats import gaussian_kde
from scipy.stats.stats import pearsonr
from scipy.optimize import curve_fit
from scipy import interp
from scipy.stats import sem
import scipy.stats as stats
import statsmodels.formula.api as sm
import statsmodels.api as sma

from collections import defaultdict

random.seed(17538)

%matplotlib inline 

direction = 'directory'
os.chdir(direction)

In [2]:
def preprocess(x, y):
    common_genes=list(set(x.index)&set(y.index))
    x_new=x.T[common_genes].T
    y_new=y.T[common_genes].T
    return x_new, y_new 

# 1. Predict categories in a multi-class model 

In [3]:
# genes which are expressed in at least 10 cells 
cate0 = pd.read_csv(direction+'categorial_distrib_0_own_new_thresh0_2.csv', index_col=0, header=None) # 6,282 genes
cate3 = pd.read_csv(direction+'categorial_distrib_3_own_new_thresh0_2.csv', index_col=0, header=None) # 6,282 genes

In [4]:
# 10 cross-validation split sets on gene level
test0 = pd.read_csv('cate0_split.tsv', sep='\t', index_col=0)
test3 = pd.read_csv('cate3_split.tsv', sep='\t', index_col=0)

In [5]:
# categories according to bulk, for day0 and day3
cate0_bulk = pd.read_csv('/homes/stlinker/paper_data/categorial_in_ex_0_own_new_thresh0_2.csv', index_col=0, header=None)
cate3_bulk = pd.read_csv('/homes/stlinker/paper_data/categorial_in_ex_3_own_new_0_2.csv', index_col=0, header=None)

In [6]:
# switching states for day0
cate0_ex = pd.read_csv('transfer_df_ex.csv', index_col=0, header=0)
cate0_in = pd.read_csv('transfer_df_in.csv', index_col=0, header=0)
cate0_mult = pd.read_csv('transfer_df_mult.csv', index_col=0, header=0)
cate0_over = pd.read_csv('transfer_df_over.csv', index_col=0, header=0)
cate0_under = pd.read_csv('transfer_df_under.csv', index_col=0, header=0)
cate0_ex.index = cate0_ex.exon
cate0_in.index = cate0_in.exon
cate0_mult.index = cate0_mult.exon
cate0_over.index = cate0_over.exon
cate0_under.index = cate0_under.exon

In [7]:
# day0 features, without and with methylation
df_features_fin=pd.read_csv(direction+'new_dataframes_features/df_no_meth_3mers_4letter.csv', index_col='gene_id') 
df_features_fin_meth=pd.read_csv(direction+'new_dataframes_features/avg_meth_6_letter_code.csv', index_col='Unnamed: 0')

In [8]:
# day3 features, without and with methylation
df_features_fin3=pd.read_csv(direction+'new_dataframes_features_3/df_no_meth_3mers_4letter.csv', index_col='gene_id')
df_features_fin_meth3=pd.read_csv(direction+'new_dataframes_features_3/avg_meth_6_letter_code.csv', index_col='Unnamed: 0')

In [11]:
# only mean methylatiion
df_features_fin_meth_columns1 = df_features_fin_meth.columns[~df_features_fin_meth.columns.isin(df_features_fin.columns)]
df_features_fin_meth_columns2 = df_features_fin_meth_columns1[0:4]
df_features_fin_meth_columns3 = df_features_fin_meth_columns1[816:len(df_features_fin_meth_columns1)]
df_features_fin_meth_columns = np.concatenate((df_features_fin_meth_columns2,df_features_fin_meth_columns3))
df_features_avmeth = df_features_fin_meth[df_features_fin_meth_columns]

df_features_fin_meth_columns1 = df_features_fin_meth3.columns[~df_features_fin_meth3.columns.isin(df_features_fin3.columns)]
df_features_fin_meth_columns = df_features_fin_meth_columns1[0:14]
df_features_avmeth3 = df_features_fin_meth3[df_features_fin_meth_columns]

In [ ]:
# choose features and categories
df_features_new, cate0_new = preprocess(df_features_avmeth, cate0)
cate0_new = pd.Series(cate0_new.iloc[:,0])

In [13]:
# choose one split set
test0 = pd.read_csv('cate0_split.tsv', sep='\t', index_col=0) #alternative: cate0_split_alt1.tsv, cate0_split_alt2.tsv, etc
test3 = pd.read_csv('cate3_split.tsv', sep='\t', index_col=0)

In [ ]:
df_features_new = df_features_new.loc[cate0_new.index]
any(df_features_new.sum()==0) # no feature should be zero, i.e. this has to be False
df_features_new = df_features_new.iloc[:,np.where(df_features_new.sum()!=0)[0]]

In [714]:
# overwrite test0 if day3 data
test0 = test3.copy()

In [ ]:
# two class prediction, e.g. for the switching state
output = 'ROC_switch_ex'
lw = 1
fig = plt.figure(facecolor='white')
ax = plt.subplot(111)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

test0 = test0[test0.exon.isin(df_features_new.index)]

# split training and test sets
for k in range(0,10):
    X_test = df_features_new.loc[test0[test0.split==k].exon]
    y_test = cate0_new.loc[test0[test0.split==k].exon]
    X_train = df_features_new.loc[test0[test0.split!=k].exon]
    y_train = cate0_new.loc[test0[test0.split!=k].exon]
    
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    classifier = sklearn.linear_model.LogisticRegression(multi_class='ovr')
    y_score = classifier.fit(X_train, y_train).decision_function(X_test)

    n_classes = 2
    
    categ = ['change', 'stay']

    y_test = label_binarize(y_test, classes=categ)
    color = 'cornflowerblue'
    
    fpr, tpr, _ = roc_curve(y_test, y_score)
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)    

# mean across cross-validations    
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)

std_auc = sem(aucs) 

plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'$\mu$AUC=%0.2f' % (mean_auc),
         lw=2, alpha=.8)

std_tpr = sem(tprs, axis=0)

tprs_upper = np.minimum(mean_tpr + 1.96 * std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - 1.96 * std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
leg = plt.legend(loc="lower right")
leg.get_frame().set_linewidth(0.0)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

In [ ]:
# get all AUCs from two class prediction and compare them for switching predictions
mean_aucs_all = pd.read_csv('final_figures/AUCs_switch.tsv', sep='\t', index_col=0)
mean_aucs_all_meth = pd.read_csv('final_figures/AUCs_switch_meth.tsv', sep='\t', index_col=0)
meanss = np.mean(mean_aucs_all, axis=1)
semss = sem(mean_aucs_all, axis=1)
meanss_meth = np.mean(mean_aucs_all_meth, axis=1)
semss_meth = sem(mean_aucs_all_meth, axis=1)

ax = plt.subplot(111)
bars = ax.bar(range(0,5),height=meanss[0:5],align='center',color='green',yerr=[1.96*semss[0:5],1.96*semss[0:5]],capsize=4)
patterns = ('\\','/','.','-', 'x')
for bar, pattern in zip(bars, patterns):
    bar.set_hatch(pattern)
    
bars = ax.bar(range(6,11),height=meanss_meth[0:5],align='center',color='blue',yerr=[1.96*semss_meth[0:5],1.96*semss_meth[0:5]],capsize=4)
patterns = ('\\','/','.','-', 'x')
for bar, pattern in zip(bars, patterns):
    bar.set_hatch(pattern)

ax.set_xticks((2,8))
ax.set_xticklabels( ('All', 'All meth') )
ax.set_ylabel(r'$\mu$AUC')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

In [ ]:
# multiclass prediction: calculate macro AUC
fpr = defaultdict(dict)
tpr = defaultdict(dict)
roc_auc = defaultdict(dict)

tprs = defaultdict(dict)
aucs = defaultdict(dict)
mean_fpr = np.linspace(0, 1, 100)

mean_tpr = defaultdict(dict)
mean_auc = defaultdict(dict)
std_auc = defaultdict(dict)
std_tpr = defaultdict(dict)
tprs_upper = defaultdict(dict)
tprs_lower = defaultdict(dict)

fprmacro = defaultdict(dict)

test0 = test0[test0.exon.isin(df_features_new.index)]

# split training and test sets
for k in range(0,10):
    X_test = df_features_new.loc[test0[test0.split==k].exon]
    y_test = cate0_new.loc[test0[test0.split==k].exon]
    X_train = df_features_new.loc[test0[test0.split!=k].exon]
    y_train = cate0_new.loc[test0[test0.split!=k].exon]
    
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    classifier = RidgeClassifier(random_state=10)
    y_score = classifier.fit(X_train, y_train).decision_function(X_test)
    
    n_classes = y_score.shape[1]

    categ = ['excluded', 'included', 'multimodal', 'overdispersed','underdispersed']
    y_test = label_binarize(y_test, classes=categ)
    
    for i in range(n_classes):
        fpr[i] = roc_curve(y_test[:, i], y_score[:, i])[0]
        tpr[i] = roc_curve(y_test[:, i], y_score[:, i])[1]
        tprs[i][k] = interp(mean_fpr, fpr[i], tpr[i])
        tprs[i][k][0] = 0.0
        roc_auc[i] = auc(fpr[i], tpr[i])
        aucs[i][k] = roc_auc[i]


for i in range(n_classes):
    mean_tpr[i] = np.mean(list(tprs[i].values()), axis=0)
    mean_tpr[i][-1] = 1.0

mean_tpr_all = np.mean(list(mean_tpr.values()), axis=0)
roc_auc_macro = auc(mean_fpr, mean_tpr_all)

In [ ]:
# multiclass prediction
lw = 1
fig = plt.figure(facecolor='white')
ax = plt.subplot(111)

fpr = defaultdict(dict)
tpr = defaultdict(dict)
roc_auc = defaultdict(dict)

tprs = defaultdict(dict)
aucs = defaultdict(dict)
mean_fpr = np.linspace(0, 1, 100)

mean_tpr = defaultdict(dict)
mean_auc = defaultdict(dict)
std_auc = defaultdict(dict)
std_tpr = defaultdict(dict)
tprs_upper = defaultdict(dict)
tprs_lower = defaultdict(dict)

fprmacro = defaultdict(dict)

test0 = test0[test0.exon.isin(df_features_new.index)]

# split training and test sets
for k in range(0,10):
    X_test = df_features_new.loc[test0[test0.split==k].exon]
    y_test = cate0_new.loc[test0[test0.split==k].exon]
    X_train = df_features_new.loc[test0[test0.split!=k].exon]
    y_train = cate0_new.loc[test0[test0.split!=k].exon]
    
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    classifier = RidgeClassifier(random_state=10)
    y_score = classifier.fit(X_train, y_train).decision_function(X_test)
    
    n_classes = y_score.shape[1]

    categ = ['excluded', 'included', 'multimodal', 'overdispersed','underdispersed']
    y_test = label_binarize(y_test, classes=categ)
    colors = ['cornflowerblue', 'crimson', 'goldenrod', 'teal', 'orchid']
    
    if k==0:
        y_test_all = y_test.copy()
        y_score_all = y_score.copy()
    else:
        y_test_all = np.append(y_test_all, y_test, axis=0)
        y_score_all = np.append(y_score_all, y_score, axis=0)
        
for i in range(n_classes):
    fpr[i] = roc_curve(y_test_all[:, i], y_score_all[:, i])[0]
    tpr[i] = roc_curve(y_test_all[:, i], y_score_all[:, i])[1]
    roc_auc[i] = auc(fpr[i], tpr[i])
    aucs[i] = roc_auc[i]
    
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i],
             label=r'%s AUC=%0.2f' % (categ[i], aucs[i]),
             lw=2, alpha=.8)

plt.plot(mean_fpr, mean_tpr_all, 'navy',
         label=r'macro AUC=%0.2f' % (roc_auc_macro),
         lw=2, alpha=.8)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
leg = plt.legend(loc="lower right")
leg.get_frame().set_linewidth(0.0)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# 2. Get most important features (in one model and in per-feature model)

In [ ]:
# two class prediction: extract most important features together with other infos
score_dict=pd.DataFrame(index=df_features_new.columns, columns = range(0,10))
auc_dict=pd.DataFrame(index=df_features_new.columns, columns = range(0,10))
weight_dict=pd.DataFrame(index=df_features_new.columns, columns = range(0,10))

for k in range(0,10):
    X_test = df_features_new.loc[test0[test0.split==k].exon]
    y_test = cate0_new.loc[test0[test0.split==k].exon]
    X_train = df_features_new.loc[test0[test0.split!=k].exon]
    y_train = cate0_new.loc[test0[test0.split!=k].exon] 
    for name in df_features_new.columns:
        X_testn = np.array(X_test[name])
        X_testn = X_testn.reshape(-1, 1)
        X_trainn = np.array(X_train[name])
        X_trainn = X_trainn.reshape(-1, 1)
        y_testn = np.array(y_test)
        y_trainn = np.array(y_train)
        classifier = RidgeClassifier(random_state=10)
        y_score = classifier.fit(X_trainn, y_trainn).decision_function(X_testn)
        n_classes = 2
        #y_testn = label_binarize(y_testn, classes=['excluded', 'included'])
        y_testn = label_binarize(y_testn, classes=['change', 'stay'])
        fpr, tpr, _ = roc_curve(y_testn, y_score)
        roc_auc = auc(fpr, tpr)
        score_dict[k].loc[name]=np.abs(np.std(df_features_new[name], 0)*classifier.coef_[:,0])
        auc_dict[k].loc[name]=roc_auc
        weight_dict[k].loc[name]=classifier.coef_[:,0]
        
i= score_dict.columns[0]

score_dict = pd.DataFrame(score_dict.mean(axis=1),columns=[i])
auc_dict = pd.DataFrame(auc_dict.mean(axis=1),columns=[i])
weight_dict = pd.DataFrame(weight_dict.mean(axis=1),columns=[i])
        
columnarray = np.array(['feature','AUC','effect','sign'])

score_df10 = pd.DataFrame(index=range(0,score_dict.shape[0]), columns=columnarray)
score_df10['feature'] = score_dict[i].loc[auc_dict.index[auc_dict[i].argsort()[::-1].values]].index
score_df10['effect'].loc[score_df10.index] = np.array([round(x,3) for x in score_dict[i].loc[auc_dict.index[auc_dict[i].argsort()[::-1].values]].values])
score_df10['sign'].loc[score_df10.index] = np.array([np.sign(x) for x in weight_dict[i].loc[auc_dict.index[auc_dict[i].argsort()[::-1].values]].values])
score_df10['AUC'].loc[score_df10.index] = np.array([round(x,3) for x in auc_dict[i].loc[auc_dict.index[auc_dict[i].argsort()[::-1].values]].values])
score_df10.index = range(1,(score_dict.shape[0] + 1))

score_df10.head()



In [ ]:
# multiclass prediction: extract most important features together with other infos

score_dict_all = defaultdict(dict)
auc_dict_all = defaultdict(dict) 
weight_dict_all = defaultdict(dict) 

for k in range(0,10):
    X_test = df_features_new.loc[test0[test0.split==k].exon]
    y_test = cate0_new.loc[test0[test0.split==k].exon]
    X_train = df_features_new.loc[test0[test0.split!=k].exon]
    y_train = cate0_new.loc[test0[test0.split!=k].exon]
    
    score_dict_all[k] = pd.DataFrame(index=df_features_new.columns, columns = ['excluded', 'included', 'multimodal', 'overdispersed', 'underdispersed'])
    auc_dict_all[k] = pd.DataFrame(index=df_features_new.columns, columns = ['excluded', 'included', 'multimodal', 'overdispersed', 'underdispersed'])
    weight_dict_all[k] = pd.DataFrame(index=df_features_new.columns, columns = ['excluded', 'included', 'multimodal', 'overdispersed', 'underdispersed'])
    
    for name in df_features_new.columns:
        X_testn = np.array(X_test[name])
        X_testn = X_testn.reshape(-1, 1)
        X_trainn = np.array(X_train[name])
        X_trainn = X_trainn.reshape(-1, 1)
        y_testn = np.array(y_test)
        y_trainn = np.array(y_train)
        classifier = RidgeClassifier(random_state=10)
        y_score = classifier.fit(X_trainn, y_trainn).decision_function(X_testn)
        n_classes = y_score.shape[1]
        
        y_testn = label_binarize(y_testn, classes=['excluded', 'included', 'multimodal', 'overdispersed', 'underdispersed'])
        
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_testn[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        score_dict_all[k].loc[name]=np.abs(np.std(df_features_new[name], 0)*classifier.coef_[:,0])
        auc_dict_all[k].loc[name]=list(roc_auc.values())
        weight_dict_all[k].loc[name]=classifier.coef_[:,0]
        
        
score_dict_alls = score_dict_all[0]
auc_dict_alls = auc_dict_all[0]
weight_dict_alls = weight_dict_all[0]
for k in range(1,10):
    score_dict_alls = pd.concat([score_dict_alls,score_dict_all[k]])
    auc_dict_alls = pd.concat([auc_dict_alls,auc_dict_all[k]])
    weight_dict_alls = pd.concat([weight_dict_alls,weight_dict_all[k]])
score_dict_alls = score_dict_alls.groupby(score_dict_alls.index).sum()/10
auc_dict_alls = auc_dict_alls.groupby(auc_dict_alls.index).sum()/10
weight_dict_alls = weight_dict_alls.groupby(weight_dict_alls.index).sum()/10

columnarray = [np.array(['excluded','excluded','excluded','excluded','included','included','included',
                         'included','multimodal','multimodal','multimodal','multimodal','overdispersed','overdispersed',
                         'overdispersed','overdispersed','underdispersed','underdispersed','underdispersed','underdispersed']),
              np.array(['feature','AUC','effect','sign','feature','AUC','effect','sign',
                       'feature','AUC','effect','sign','feature','AUC','effect','sign',
                       'feature','AUC','effect','sign'])]


score_df10 = pd.DataFrame(index=score_dict_alls.index, columns=columnarray)
for i in score_dict_alls.columns:
    score_df10[i]['feature'] = score_dict_alls[i].loc[auc_dict_alls.index[auc_dict_alls[i].argsort()[::-1].values]].index
    score_df10[i]['effect'].loc[score_df10.index] = np.array([round(x,3) for x in score_dict_alls[i].loc[auc_dict_alls.index[auc_dict_alls[i].argsort()[::-1].values]].values])
    score_df10[i]['sign'].loc[score_df10.index] = np.array([np.sign(x) for x in weight_dict_alls[i].loc[auc_dict_alls.index[auc_dict_alls[i].argsort()[::-1].values]].values])
    score_df10[i]['AUC'].loc[score_df10.index] = np.array([round(x,3) for x in auc_dict_alls[i].loc[auc_dict_alls.index[auc_dict_alls[i].argsort()[::-1].values]].values])
score_df10.index = range(1,(score_dict_alls.shape[0]+1))

score_df10.head()